In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import DataClass as dc

# Passo 1: Carregar os datasets e converter os valores não numéricos para valores numéricos

dt2 = pd.read_csv(dc.CSV_PATH_2)
dt5 = pd.read_csv(dc.CSV_PATH_5)
dt7 = pd.read_csv(dc.CSV_PATH_7)

dt = pd.concat([dt2, dt5, dt7], ignore_index=True)

pd.set_option('display.max_columns', None)

# Identificar colunas não numéricas
non_numeric_columns = dt.select_dtypes(include=['object']).columns

# Inicializar o LabelEncoder
label_encoder = LabelEncoder()

# Converter colunas não numéricas para valores numéricos
for column in non_numeric_columns:
    dt[column] = label_encoder.fit_transform(dt[column])

print(dt.head())

      Id  Altitude  Slope_Orientation  Slope  Horizontal_Distance_To_Water  \
0  14537      3166                198      6                           192   
1   1154      2877                342     13                           450   
2  14507      3103                 48      7                             0   
3   7196      2950                269     39                           450   
4   1801      2868                332     17                            60   

   Vertical_Distance_To_Water  Horizontal_Distance_To_Roadways  \
0                          -4                             3137   
1                         131                              618   
2                           0                             1034   
3                         316                             1528   
4                          23                             1718   

   Shadow_Index_9h  Shadow_Index_12h  Shadow_Index_15h  \
0              218               245               163   
1              193

In [6]:
# Passo 2: Univariate Analysis (Distribution of individual features)
## variável Altitude

mean_altitude = dt[dc.ALTITUDE].mean()
median_altitude = dt[dc.ALTITUDE].median()
mode_altitude = dt[dc.ALTITUDE].mode()[0]
std_dev_altitude = dt[dc.ALTITUDE].std()
range_altitude = dt[dc.ALTITUDE].max() - dt[dc.ALTITUDE].min()
variance_altitude = dt[dc.ALTITUDE].var()
iqr_altitude = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Altitude')
print(f'Mean: {mean_altitude}')
print(f'Median: {median_altitude}')
print(f'Mode: {mode_altitude}')
print(f'Standard Deviation: {std_dev_altitude}')
print(f'Range: {range_altitude}')
print(f'Variance: {variance_altitude}')
print(f'IQR: {iqr_altitude}')


## variável Slope_orientation

mean_slope_orientation = dt[dc.SLOPE_ORIENTATION].mean()
median_slope_orientation = dt[dc.SLOPE_ORIENTATION].median()
mode_slope_orientation = dt[dc.SLOPE_ORIENTATION].mode()[0]
std_dev_slope_orientation = dt[dc.SLOPE_ORIENTATION].std()
range_slope_orientation = dt[dc.SLOPE_ORIENTATION].max() - dt[dc.SLOPE_ORIENTATION].min()
variance_slope_orientation = dt[dc.SLOPE_ORIENTATION].var()
iqr_slope_orientation = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Slope_orientation')
print(f'Mean: {mean_slope_orientation}')
print(f'Median: {median_slope_orientation}')
print(f'Mode: {mode_slope_orientation}')
print(f'Standard Deviation: {std_dev_slope_orientation}')
print(f'Range: {range_slope_orientation}')
print(f'Variance: {variance_slope_orientation}')
print(f'IQR: {iqr_slope_orientation}')

## variável Slope

mean_slope = dt[dc.SLOPE].mean()
median_slope = dt[dc.SLOPE].median()
mode_slope = dt[dc.SLOPE].mode()[0]
std_dev_slope = dt[dc.SLOPE].std()
range_slope = dt[dc.SLOPE].max() - dt[dc.SLOPE].min()
variance_slope = dt[dc.SLOPE].var()
iqr_slope = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Slope')
print(f'Mean: {mean_slope}')
print(f'Median: {median_slope}')
print(f'Mode: {mode_slope}')
print(f'Standard Deviation: {std_dev_slope}')
print(f'Range: {range_slope}')
print(f'Variance: {variance_slope}')
print(f'IQR: {iqr_slope}')

## variável Horizontal_Distance_To_Water

mean_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].mean()
median_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].median()
mode_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].mode()[0]
std_dev_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].std()
range_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].max() - dt[dc.HORIZONTAL_DISTANCE_TO_WATER].min()
variance_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_WATER].var()
iqr_horizontal_distance_to_water = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Horizontal_Distance_To_Water')
print(f'Mean: {mean_horizontal_distance_to_water}')
print(f'Median: {median_horizontal_distance_to_water}')
print(f'Mode: {mode_horizontal_distance_to_water}')
print(f'Standard Deviation: {std_dev_horizontal_distance_to_water}')
print(f'Range: {range_horizontal_distance_to_water}')
print(f'Variance: {variance_horizontal_distance_to_water}')
print(f'IQR: {iqr_horizontal_distance_to_water}')

## variável VERTICAL_DISTANCE_TO_WATER

mean_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].mean()
median_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].median()
mode_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].mode()[0]
std_dev_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].std()
range_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].max() - dt[dc.VERTICAL_DISTANCE_TO_WATER].min()
variance_vertical_distance_to_water = dt[dc.VERTICAL_DISTANCE_TO_WATER].var()
iqr_vertical_distance_to_water = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Vertical_Distance_To_Water')
print(f'Mean: {mean_vertical_distance_to_water}')
print(f'Median: {median_vertical_distance_to_water}')
print(f'Mode: {mode_vertical_distance_to_water}')
print(f'Standard Deviation: {std_dev_vertical_distance_to_water}')
print(f'Range: {range_vertical_distance_to_water}')
print(f'Variance: {variance_vertical_distance_to_water}')
print(f'IQR: {iqr_vertical_distance_to_water}')

## variável HORIZONTAL_DISTANCE_TO_ROADWAYS

mean_horizontal_distance_to_roadways = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].mean()
median_horizontal_distance_to_roadways = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].median()
mode_horizontal_distance_to_roadways = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].mode()[0]
std_dev_horizontal_distance_to_roadways = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].std()
range_horizontal_distance_to_water = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].max() - dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].min()
variance_horizontal_distance_to_roadways = dt[dc.HORIZONTAL_DISTANCE_TO_ROADWAYS].var()
iqr_horizontal_distance_to_roadways = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Horizontal_Distance_To_Roadways')
print(f'Mean: {mean_horizontal_distance_to_roadways}')
print(f'Median: {median_horizontal_distance_to_roadways}')
print(f'Mode: {mode_horizontal_distance_to_roadways}')
print(f'Standard Deviation: {std_dev_horizontal_distance_to_roadways}')
print(f'Range: {range_horizontal_distance_to_water}')
print(f'Variance: {variance_horizontal_distance_to_roadways}')
print(f'IQR: {iqr_horizontal_distance_to_roadways}')

## variável SHADOW_INDEX_9

mean_shadow_index_9 = dt[dc.SHADOW_INDEX_9].mean()
median_shadow_index_9 = dt[dc.SHADOW_INDEX_9].median()
mode_shadow_index_9 = dt[dc.SHADOW_INDEX_9].mode()[0]
std_dev_shadow_index_9 = dt[dc.SHADOW_INDEX_9].std()
range_shadow_index_9 = dt[dc.SHADOW_INDEX_9].max() - dt[dc.SHADOW_INDEX_9].min()
variance_shadow_index_9 = dt[dc.SHADOW_INDEX_9].var()
iqr_shadow_index_9 = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Shadow_Index_9')
print(f'Mean: {mean_shadow_index_9}')
print(f'Median: {median_shadow_index_9}')
print(f'Mode: {mode_shadow_index_9}')
print(f'Standard Deviation: {std_dev_shadow_index_9}')
print(f'Range: {range_shadow_index_9}')
print(f'Variance: {variance_shadow_index_9}')
print(f'IQR: {iqr_shadow_index_9}')

## variável SHADOW_INDEX_12

mean_shadow_index_12 = dt[dc.SHADOW_INDEX_12].mean()
median_shadow_index_12 = dt[dc.SHADOW_INDEX_12].median()
mode_shadow_index_12 = dt[dc.SHADOW_INDEX_12].mode()[0]
std_dev_shadow_index_12 = dt[dc.SHADOW_INDEX_12].std()
range_shadow_index_12 = dt[dc.SHADOW_INDEX_12].max() - dt[dc.SHADOW_INDEX_12].min()
variance_shadow_index_12 = dt[dc.SHADOW_INDEX_12].var()
iqr_shadow_index_12 = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Shadow_Index_12')
print(f'Mean: {mean_shadow_index_12}')
print(f'Median: {median_shadow_index_12}')
print(f'Mode: {mode_shadow_index_12}')
print(f'Standard Deviation: {std_dev_shadow_index_12}')
print(f'Range: {range_shadow_index_12}')
print(f'Variance: {variance_shadow_index_12}')
print(f'IQR: {iqr_shadow_index_12}')

## variável SHADOW_INDEX_15

mean_shadow_index_15 = dt[dc.SHADOW_INDEX_15].mean()
median_shadow_index_15 = dt[dc.SHADOW_INDEX_15].median()
mode_shadow_index_15 = dt[dc.SHADOW_INDEX_15].mode()[0]
std_dev_shadow_index_15 = dt[dc.SHADOW_INDEX_15].std()
range_shadow_index_15 = dt[dc.SHADOW_INDEX_15].max() - dt[dc.SHADOW_INDEX_15].min()
variance_shadow_index_15 = dt[dc.SHADOW_INDEX_15].var()
iqr_shadow_index_15 = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Shadow_Index_15')
print(f'Mean: {mean_shadow_index_15}')
print(f'Median: {median_shadow_index_15}')
print(f'Mode: {mode_shadow_index_15}')
print(f'Standard Deviation: {std_dev_shadow_index_15}')
print(f'Range: {range_shadow_index_15}')
print(f'Variance: {variance_shadow_index_15}')
print(f'IQR: {iqr_shadow_index_15}')

## variável HORIZONTAL_DISTANCE_TO_FIRE_POINTS

mean_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].mean()
median_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].median()
mode_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].mode()[0]
std_dev_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].std()
range_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].max() - dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].min()
variance_horizontal_distance_to_fire_points = dt[dc.HORIZONTAL_DISTANCE_TO_FIRE_POINTS].var()
iqr_horizontal_distance_to_fire_points = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Horizontal_Distance_To_Fire_Points')
print(f'Mean: {mean_horizontal_distance_to_fire_points}')
print(f'Median: {median_horizontal_distance_to_fire_points}')
print(f'Mode: {mode_horizontal_distance_to_fire_points}')
print(f'Standard Deviation: {std_dev_horizontal_distance_to_fire_points}')
print(f'Range: {range_horizontal_distance_to_fire_points}')
print(f'Variance: {variance_horizontal_distance_to_fire_points}')
print(f'IQR: {iqr_horizontal_distance_to_fire_points}')

## variável CANOPY_DENSITY

mean_canopy_density = dt[dc.CANOPY_DENSITY].mean()
median_canopy_density = dt[dc.CANOPY_DENSITY].median()
mode_canopy_density = dt[dc.CANOPY_DENSITY].mode()[0]
std_dev_canopy_density = dt[dc.CANOPY_DENSITY].std()
range_canopy_density = dt[dc.CANOPY_DENSITY].max() - dt[dc.CANOPY_DENSITY].min()
variance_canopy_density = dt[dc.CANOPY_DENSITY].var()
iqr_canopy_density = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Canopy_Density')
print(f'Mean: {mean_canopy_density}')
print(f'Median: {median_canopy_density}')
print(f'Mode: {mode_canopy_density}')
print(f'Standard Deviation: {std_dev_canopy_density}')
print(f'Range: {range_canopy_density}')
print(f'Variance: {variance_canopy_density}')

## variável RAINFALL_SUMMER

mean_rainfall_summer = dt[dc.RAINFALL_SUMMER].mean()
median_rainfall_summer = dt[dc.RAINFALL_SUMMER].median()
mode_rainfall_summer = dt[dc.RAINFALL_SUMMER].mode()[0]
std_dev_rainfall_summer = dt[dc.RAINFALL_SUMMER].std()
range_rainfall_summer = dt[dc.RAINFALL_SUMMER].max() - dt[dc.RAINFALL_SUMMER].min()
variance_rainfall_summer = dt[dc.RAINFALL_SUMMER].var()
iqr_rainfall_summer = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Rainfall_Summer')
print(f'Mean: {mean_rainfall_summer}')
print(f'Median: {median_rainfall_summer}')
print(f'Mode: {mode_rainfall_summer}')
print(f'Standard Deviation: {std_dev_rainfall_summer}')
print(f'Range: {range_rainfall_summer}')
print(f'Variance: {variance_rainfall_summer}')
print(f'IQR: {iqr_rainfall_summer}')

## variável RAINFALL_WINTER

mean_rainfall_winter = dt[dc.RAINFALL_WINTER].mean()
median_rainfall_winter = dt[dc.RAINFALL_WINTER].median()
mode_rainfall_winter = dt[dc.RAINFALL_WINTER].mode()[0]
std_dev_rainfall_winter = dt[dc.RAINFALL_WINTER].std()
range_rainfall_winter = dt[dc.RAINFALL_WINTER].max() - dt[dc.RAINFALL_WINTER].min()
variance_rainfall_winter = dt[dc.RAINFALL_WINTER].var()
iqr_rainfall_winter = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Rainfall_Winter')
print(f'Mean: {mean_rainfall_winter}')
print(f'Median: {median_rainfall_winter}')
print(f'Mode: {mode_rainfall_winter}')
print(f'Standard Deviation: {std_dev_rainfall_winter}')
print(f'Range: {range_rainfall_winter}')
print(f'Variance: {variance_rainfall_winter}')
print(f'IQR: {iqr_rainfall_winter}')

## variável WIND_EXPOSURE_LEVEL

mean_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].mean()
median_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].median()
mode_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].mode()[0]
std_dev_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].std()
range_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].max() - dt[dc.WIND_EXPOSURE_LEVEL].min()
variance_wind_exposure_level = dt[dc.WIND_EXPOSURE_LEVEL].var()
iqr_wind_exposure_level = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Wind_Exposure_Level')
print(f'Mean: {mean_wind_exposure_level}')
print(f'Median: {median_wind_exposure_level}')
print(f'Mode: {mode_wind_exposure_level}')
print(f'Standard Deviation: {std_dev_wind_exposure_level}')
print(f'Range: {range_wind_exposure_level}')
print(f'Variance: {variance_wind_exposure_level}')
print(f'IQR: {iqr_wind_exposure_level}')

## variável SOIL_TYPE

mean_soil_type = dt[dc.SOIL_TYPE].mean()
median_soil_type = dt[dc.SOIL_TYPE].median()
mode_soil_type = dt[dc.SOIL_TYPE].mode()[0]
std_dev_soil_type = dt[dc.SOIL_TYPE].std()
range_soil_type = dt[dc.SOIL_TYPE].max() - dt[dc.SOIL_TYPE].min()
variance_soil_type = dt[dc.SOIL_TYPE].var()
iqr_soil_type = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Soil_Type')
print(f'Mean: {mean_soil_type}')
print(f'Median: {median_soil_type}')
print(f'Mode: {mode_soil_type}')
print(f'Standard Deviation: {std_dev_soil_type}')
print(f'Range: {range_soil_type}')
print(f'Variance: {variance_soil_type}')
print(f'IQR: {iqr_soil_type}')

## variável WILDERNESS_AREA

mean_wilder_area = dt[dc.WILDERNESS_AREA].mean()
median_wilder_area = dt[dc.WILDERNESS_AREA].median()
mode_wilder_area = dt[dc.WILDERNESS_AREA].mode()[0]
std_dev_wilder_area = dt[dc.WILDERNESS_AREA].std()
range_wilder_area = dt[dc.WILDERNESS_AREA].max() - dt[dc.WILDERNESS_AREA].min()
variance_wilder_area = dt[dc.WILDERNESS_AREA].var()
iqr_wilder_area = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Wilderness_Area')
print(f'Mean: {mean_wilder_area}')
print(f'Median: {median_wilder_area}')
print(f'Mode: {mode_wilder_area}')
print(f'Standard Deviation: {std_dev_wilder_area}')
print(f'Range: {range_wilder_area}')
print(f'Variance: {variance_wilder_area}')
print(f'IQR: {iqr_wilder_area}')

## variável VEGETATION_TYPE

mean_vegetation_type = dt[dc.VEGETATION_TYPE].mean()
median_vegetation_type = dt[dc.VEGETATION_TYPE].median()
mode_vegetation_type = dt[dc.VEGETATION_TYPE].mode()[0]
std_dev_vegetation_type = dt[dc.VEGETATION_TYPE].std()
range_vegetation_type = dt[dc.VEGETATION_TYPE].max() - dt[dc.VEGETATION_TYPE].min()
variance_vegetation_type = dt[dc.VEGETATION_TYPE].var()
iqr_vegetation_type = dt.quantile(0.75) - dt.quantile(0.25)

print('Univariate Analysis for Vegetation_Type')
print(f'Mean: {mean_vegetation_type}')
print(f'Median: {median_vegetation_type}')
print(f'Mode: {mode_vegetation_type}')
print(f'Standard Deviation: {std_dev_vegetation_type}')
print(f'Range: {range_vegetation_type}')
print(f'Variance: {variance_vegetation_type}')
print(f'IQR: {iqr_vegetation_type}')

Univariate Analysis for Altitude
Mean: 2948.4553656220323
Median: 2873.0
Mode: 2820
Standard Deviation: 256.62249393872423
Range: 1612
Variance: 65855.10439533056
IQR: Id                                    8864.7500
Altitude                               365.2500
Slope_Orientation                      150.0000
Slope                                   11.0000
Horizontal_Distance_To_Water           271.0000
Vertical_Distance_To_Water              74.0000
Horizontal_Distance_To_Roadways       1807.5000
Shadow_Index_9h                         31.0000
Shadow_Index_12h                        27.0000
Shadow_Index_15h                        55.2500
Horizontal_Distance_To_Fire_Points    1303.0000
Canopy_Density                          29.7650
Rainfall_Summer                        148.0575
Rainfall_Winter                        555.6900
Wind_Exposure_Level                     29.4700
Soil_Type                               14.2500
Wilderness_Area                          2.0000
Vegetation_Type 